In [1]:
import pandas as pd
import math
import numpy as np
import sklearn as sk
import sklearn.datasets as skd
import sklearn.ensemble as ske
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from pprint import pprint
import random
import xlsxwriter
from openpyxl.styles import Font
from openpyxl import Workbook

In [28]:
df_weather = pd.read_excel('Weather data.xlsx', sheet_name = 'Weather')
df_weather = df_weather.rename(columns={'ppt (mm)':'ppt','tmin (degrees C)':'tmin','tmean (degrees C)':'tmean',
                                        'tmax (degrees C)':'tmax', 'tdmean (degrees C)':'tdew', 'vpdmin (hPa)': 'vpdmin', 
                                        'vpdmax (hPa)': 'vpdmax'})

df_weather['Year'] = pd.DatetimeIndex(df_weather['Date']).year
df_weather['Month'] = pd.DatetimeIndex(df_weather['Date']).month
df_weather['Day'] = pd.DatetimeIndex(df_weather['Date']).day
df_weather['Day of year'] = df_weather['Date'].dt.dayofyear

df_weather.head()

,Date,ppt,tmin,tmean,tmax,tdew,vpdmin,vpdmax,Year,Month,Day,Day of year
0,1981-01-01,0.0,-2.0,1.3,4.6,-2.1,0.56,2.91,1981,1,1,1
1,1981-01-02,0.0,-11.2,-4.7,1.9,-8.1,0.27,3.29,1981,1,2,2
2,1981-01-03,0.0,-14.6,-4.6,5.5,-10.3,0.33,5.59,1981,1,3,3
3,1981-01-04,0.0,-15.1,-12.9,-10.7,-21.6,0.70,1.69,1981,1,4,4
4,1981-01-05,0.0,-13.5,-10.5,-7.6,-20.9,1.01,2.39,1981,1,5,5


In [29]:
df_annual_ppt = df_weather.groupby(['Year'])['ppt'].sum()
df_annual_ppt = df_annual_ppt.to_frame().reset_index()
df_annual_ppt = df_annual_ppt.rename(columns={'ppt':'Annual ppt'})
df_annual_ppt['Prev annual ppt'] = df_annual_ppt['Annual ppt'].shift(1)
df_annual_ppt.tail()

,Year,Annual ppt,Prev annual ppt
35,2016,869.16,1365.19
36,2017,861.04,869.16
37,2018,1038.78,861.04
38,2019,985.64,1038.78
39,2020,557.18,985.64


In [30]:
df_weather = pd.merge(df_weather, df_annual_ppt, left_on='Year', right_on='Year', how='left')
#df_merged = df_merged.dropna()

df_weather.tail()

,Date,ppt,tmin,tmean,tmax,tdew,vpdmin,vpdmax,Year,Month,Day,Day of year,Annual ppt,Prev annual ppt
14605,2020-12-27,0.00,-11.0,-5.1,0.7,-5.2,0.08,1.49,2020,12,27,362,557.18,985.64
14606,2020-12-28,0.60,-9.0,-5.1,-1.2,-5.1,0.14,0.58,2020,12,28,363,557.18,985.64
14607,2020-12-29,0.00,-11.9,-7.8,-3.6,-9.4,0.07,1.28,2020,12,29,364,557.18,985.64
14608,2020-12-30,5.62,-7.7,-5.3,-2.9,-5.3,0.00,0.65,2020,12,30,365,557.18,985.64
14609,2020-12-31,0.00,-13.4,-8.8,-4.3,-8.8,0.06,0.98,2020,12,31,366,557.18,985.64


In [20]:
df_weather['Annual ppt'] = df_weather['ppt'].groupby(df_weather['Year']).transform('sum')
df_weather['Prev annual ppt'] = df_weather['Annual ppt'].shift(1)

df_weather.tail()

,Date,ppt,tmin,tmean,tmax,tdew,vpdmin,vpdmax,Year,Month,Day,Day of year,Annual ppt,Prev annual ppt
14605,2020-12-27,0.00,-11.0,-5.1,0.7,-5.2,0.08,1.49,2020,12,27,362,557.18,557.18
14606,2020-12-28,0.60,-9.0,-5.1,-1.2,-5.1,0.14,0.58,2020,12,28,363,557.18,557.18
14607,2020-12-29,0.00,-11.9,-7.8,-3.6,-9.4,0.07,1.28,2020,12,29,364,557.18,557.18
14608,2020-12-30,5.62,-7.7,-5.3,-2.9,-5.3,0.00,0.65,2020,12,30,365,557.18,557.18
14609,2020-12-31,0.00,-13.4,-8.8,-4.3,-8.8,0.06,0.98,2020,12,31,366,557.18,557.18
